# Distances and trip durations with google maps API

In [1]:
import json
import googlemaps
import numpy as np
import pandas as pd

from shapely import wkt
from datetime import datetime

## Loading the data

In [2]:
hospitals = pd.read_csv('./data/mainhospitals.csv')

In [4]:
cityblock = pd.read_csv('data/maindata.csv')

In [3]:
hospitals_list = [(coord.y, coord.x) for coord in [wkt.loads(point) for point in hospitals.point]]

In [5]:
cityblock['centroid'] = [wkt.loads(point) for point in cityblock.centroid]

## API calls

In [ ]:
gmaps = googlemaps.Client(key='Your Key Here')

In [ ]:
# MODIFY THIS IF THIS CODE IS OLDER THAN THE DATE BELOW
when = datetime(2018, 11, 27, 8, 0, 0)

In [ ]:
# DO NOT GIT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1

for i, block in cityblock.iterrows():
    
    target = (block.centroid.y, block.centroid.x)
    
    try:
        response = gmaps.distance_matrix(target, hospitals_list, mode='driving', departure_time=when)
        response = {'status': 'OK', 'id': block.id, 'response': response}
    except:
        print(block.tag)
        response = {'status': 'ERROR', 'id': block.id, 'response': 'empty'}
    
    with open(f'data/APIresponse/{block.tag}.json', 'w') as file:
        json.dump(response, file, ensure_ascii=False)    

## Data exploration

In [6]:
hospitals_array = hospitals.id.tolist()

In [7]:
for i, block in cityblock.iterrows():
    index = hospitals_array.index(block.area_id)
    with open(f'./data/APIresponse/{block.tag}.json', 'r') as file:
        content = json.load(file)
    times = content['response']['rows'][0]['elements']
    cityblock.loc[i, 'area_time'] = times[index]['duration']['value']
    cityblock.loc[i, 'area_distance'] = times[index]['distance']['value']/1000
    
    # index = hospitals_array.index(block.nearest_id)
    # with open(f'./data/APIresponse/{block.tag}.json', 'r') as file:
    #    content = json.load(file)
    # times = content['response']['rows'][0]['elements']
    # cityblock.loc[i, 'nearest_time'] = times[index]['duration']['value']

cityblock['area_time'] = cityblock.area_time.astype(int)
# cityblock['nearest_time'] = cityblock.nearest_time.astype(int)

In [9]:
for i, block in cityblock.iterrows():
    
    with open(f'./data/APIresponse/{block.tag}.json', 'r') as file:
        content = json.load(file)
    
    times = content['response']['rows'][0]['elements']

    fastest = min(times, key=lambda x: x['duration']['value'])

    which = times.index(fastest)
    cityblock.loc[i, 'fastest_id'] = int(hospitals_array[which])
    cityblock.loc[i, 'fastest_distance'] = fastest['distance']['value']/1000
    cityblock.loc[i, 'fastest_time'] = fastest['duration']['value']
    
cityblock['fastest_id'] = cityblock.fastest_id.astype(int)
cityblock['fastest_time'] = cityblock.fastest_time.astype(int)

In [13]:
columns_order = ['id', 'tag', 'population', 'geometry', 'centroid',
                 'area_id', 'area_distance', 'area_time',
                 'fastest_id', 'fastest_distance', 'fastest_time']

cityblock = cityblock.loc[:, columns_order]

In [15]:
cityblock.to_csv('data/maindata.csv', index=False)